second version of training that uses aggregation mlp

In [1]:
from pykeen.pipeline import pipeline
from pykeen.datasets import Nations, get_dataset
import torch
from pykeen.evaluation import evaluate, RankBasedEvaluator
from pykeen.metrics.ranking import HitsAtK
import pandas as pd


import logging
from pathlib import Path

import click
import more_click
import torch
from pykeen.evaluation import RankBasedEvaluator
from pykeen.losses import NSSALoss,CrossEntropyLoss
from pykeen.models.inductive import InductiveNodePiece, InductiveNodePieceGNN
from pykeen.trackers import ConsoleResultTracker, WANDBResultTracker, FileResultTracker
from pykeen.training import SLCWATrainingLoop
from pykeen.typing import TESTING, TRAINING, VALIDATION
from pykeen.utils import resolve_device, set_random_seed
from torch.optim import Adam


from pykeen.metrics.ranking import HitsAtK

from pathlib import Path

from pykeen.datasets.inductive.base import DisjointInductivePathDataset
from pykeen.datasets.base import PathDataset, Dataset
from typing_extensions import Literal
import os
from pykeen.hpo import hpo_pipeline
from pykeen.triples import TriplesFactory
from pykeen.models import InductiveNodePiece, TransE, RGCN, NodePiece
from pykeen.typing import TESTING, TRAINING, VALIDATION

import time

import platform

import sys

import cpuinfo

import psutil

import subprocess

import zipfile

seed = 1234

In [2]:
def show_metrics(dictionary,model_name,csv_name):
    for key in dictionary.keys():
        print(key)
        df = pd.DataFrame(dictionary[key])
        df.to_csv(f"{model_name}/{model_name}_{csv_name}_{key}.csv")
        print(df)

In [3]:
DATA_TYPE = "_transductive.tsv"
TRAIN_PATH = "MSCallGraph_train" + DATA_TYPE
TEST_PATH = "MSCallGraph_test" + DATA_TYPE
VALIDATE_PATH = "MSCallGraph_validation" + DATA_TYPE

In [4]:
dataset = PathDataset(training_path = TRAIN_PATH,
                     testing_path = TEST_PATH,
                     validation_path = VALIDATE_PATH,
                     eager = True,
                     create_inverse_triples = True
                     )


In [5]:
model_name = 'nodepiece_transductive_2'

In [6]:
tracker = ConsoleResultTracker()

In [7]:
loss = NSSALoss() #used by RotatE and NodePiece
embedding_dim = 200
num_tokens = 20

In [8]:
model = NodePiece(
        triples_factory=dataset.training,
        random_seed = seed,
        loss = loss,
        embedding_dim = embedding_dim,
        aggregation = 'mlp'
    ).to(resolve_device())
print(f"Number of parameters: {sum(p.numel() for p in model.parameters())}")
print(f"Space occupied: {model.num_parameter_bytes} bytes")

sampling:   0%|          | 0.00/16.7k [00:00<?, ?it/s]

No symbolic computation of output shape.


Number of parameters: 243200
Space occupied: 972800 bytes


In [9]:
directory = model_name

if not os.path.exists(directory):
    os.makedirs(directory)
    print(f'Directory {directory} created successfully!')
else:
    print(f'Directory {directory} already exists.')

Directory nodepiece_transductive_2 already exists.


In [10]:
learning_rate = 1e-3
optimizer = Adam(params=model.parameters(), lr=learning_rate)
num_epochs = 2000
patience = 20

In [11]:
metrics = ['meanreciprocalrank', HitsAtK(1),
                 HitsAtK(3), HitsAtK(5), HitsAtK(10)]

train_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
valid_evaluator = RankBasedEvaluator(
        metrics=metrics,
        add_defaults=False,
    )
test_evaluator = RankBasedEvaluator(
        metrics = metrics,
        add_defaults=False
    )

In [12]:
from pykeen.stoppers import EarlyStopper

stopper = EarlyStopper(
    model = model,
    metric='meanreciprocalrank',
    patience=patience,
    frequency=1,
    evaluator = valid_evaluator,
    training_triples_factory = dataset.training,
    evaluation_triples_factory = dataset.validation,
    result_tracker = tracker

)



In [13]:
# default training regime is negative sampling (SLCWA)
# you can also use the 1-N regime with the LCWATrainingLoop
# the LCWA loop does not need negative sampling kwargs, but accepts label_smoothing in the .train() method
training_loop = SLCWATrainingLoop(
        triples_factory=dataset.training,
        model=model,
        result_tracker=tracker,
        optimizer=optimizer
)

In [14]:
training_start = time.time()
train_epoch =  training_loop.train(
        triples_factory=dataset.training,
        num_epochs=num_epochs,
        stopper = stopper
        
    )
training_duration = time.time() - training_start

Training epochs on cuda:0:   0%|          | 0/2000 [00:00<?, ?epoch/s]

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 1
Metric: loss = 3.439111351966858
Step: 1
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.003948843163723198
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.014801754029375619
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.009375298596549408
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0012882475275546312
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.003920705523341894
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.002604476409032941
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0010568237886919812
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.002891990451398521
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.001974407120045251
Metric: validation.head.optimistic.hits_at_1 = 0.0020452221338484265
Metric: validation.tail.optimistic.hits_at_1 = 0.01215770935121009
Metric: validation.both.optimistic.hits_at_1 = 0

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 2
Metric: loss = 1.7329641580581665
Step: 2
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.004552139534810397
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.011182661089426869
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.007867400312118633
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0009472189121879638
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.003243145765736699
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.002095182426273823
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0007553536014482359
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.002502164653163317
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0016287591273057764
Metric: validation.head.optimistic.hits_at_1 = 0.0029542097488921715
Metric: validation.tail.optimistic.hits_at_1 = 0.005113055334621066
Metric: validation.both.optimistic.hits_at_1 

...

Step: 65
Metric: loss = 0.37104305624961853
Step: 65
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.6606721452016558
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.032911563642424445
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3467918544220401
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.0026403607334941626
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.006696356926113368
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.00466835917904973
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0018692232709265073
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.005194607276155626
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0035319152735410666
Metric: validation.head.optimistic.hits_at_1 = 0.6578797863879104
Metric: validation.tail.optimistic.hits_at_1 = 0.024088171798659245
Metric: validation.both.optimistic.hits_at_1 = 0.3

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 66
Metric: loss = 0.3699842393398285
Step: 66
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.6562245619718146
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.04452310175940157
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.35037383186560805
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.002467093989253044
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.007104374002665281
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.004785733763128519
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.0017594028076436024
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.005154896616903511
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0034571497122735575
Metric: validation.head.optimistic.hits_at_1 = 0.6534484717645722
Metric: validation.tail.optimistic.hits_at_1 = 0.037609362572434954
Metric: validation.both.optimistic.hits_at_1 = 0.34

Training batches on cuda:0:   0%|          | 0/2 [00:00<?, ?batch/s]

Step: 67
Metric: loss = 0.37441781163215637
Step: 67
Metric: validation.head.optimistic.inverse_harmonic_mean_rank = 0.6528592275877407
Metric: validation.tail.optimistic.inverse_harmonic_mean_rank = 0.047935716660345046
Metric: validation.both.optimistic.inverse_harmonic_mean_rank = 0.3503974721240429
Metric: validation.head.realistic.inverse_harmonic_mean_rank = 0.002484746742993593
Metric: validation.tail.realistic.inverse_harmonic_mean_rank = 0.006280491594225168
Metric: validation.both.realistic.inverse_harmonic_mean_rank = 0.004382619168609381
Metric: validation.head.pessimistic.inverse_harmonic_mean_rank = 0.001782878825111749
Metric: validation.tail.pessimistic.inverse_harmonic_mean_rank = 0.004587201033056262
Metric: validation.both.pessimistic.inverse_harmonic_mean_rank = 0.0031850399290840052
Metric: validation.head.optimistic.hits_at_1 = 0.6496988978525168
Metric: validation.tail.optimistic.hits_at_1 = 0.0385183501874787
Metric: validation.both.optimistic.hits_at_1 = 0.3441

In [15]:
torch.save(model,f"{model_name}/model.pth")
model = torch.load(f"{model_name}/model.pth")

In [16]:
print("Train error per epoch:")
df = pd.DataFrame(train_epoch)
print(df)
df.to_csv(f"{model_name}/{model_name}_train_error_per_epoch.csv")

Train error per epoch:
           0
0   3.439111
1   1.732964
2   1.280166
3   1.052413
4   0.901701
..       ...
62  0.367519
63  0.370390
64  0.371043
65  0.369984
66  0.374418

[67 rows x 1 columns]


In [17]:
training_evaluation_start = time.time()
# train
print("Train error")
show_metrics(train_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.training.mapped_triples,
        additional_filter_triples=[
        dataset.training.mapped_triples,
    ]
    ).to_dict(),model_name,'train_metrics')
training_evaluation_duration = time.time() - training_evaluation_start

Train error


Evaluating on cuda:0:   0%|          | 0.00/35.6k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.694646   0.002415     0.001804
hits_at_1                     0.692399   0.000309     0.000309
hits_at_3                     0.694002   0.001012     0.000619
hits_at_5                     0.694395   0.001294     0.001266
hits_at_10                    0.695998   0.003178     0.002109
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.066440   0.007371     0.005062
hits_at_1                     0.056245   0.000056     0.000056
hits_at_3                     0.066003   0.000900     0.000787
hits_at_5                     0.066369   0.010012     0.001097
hits_at_10                    0.084254   0.011249     0.011052
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.380543   0.004893     0.003433
hits_at_1                     0.374322   0.000183     0.000183
hits_at_3                     0.380002  

In [18]:
validation_evaluation_start = time.time()
# validation
print("Validation error")
show_metrics(valid_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.validation.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples
        ],
    ).to_dict(),model_name,'validation_metrics')
validation_evaluation_duration = time.time() - validation_evaluation_start

Validation error


Evaluating on cuda:0:   0%|          | 0.00/8.80k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.659897   0.002719     0.001972
hits_at_1                     0.657425   0.000227     0.000227
hits_at_3                     0.659584   0.000909     0.000454
hits_at_5                     0.659698   0.001250     0.001250
hits_at_10                    0.661175   0.003181     0.001704
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.054892   0.007470     0.005245
hits_at_1                     0.043972   0.000227     0.000227
hits_at_3                     0.055562   0.001136     0.000909
hits_at_5                     0.056016   0.012158     0.001363
hits_at_10                    0.072151   0.013180     0.012612
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.357394   0.005094     0.003608
hits_at_1                     0.350699   0.000227     0.000227
hits_at_3                     0.357573  

In [19]:
testing_evaluation_start = time.time()
# result on the test set
print("Test error")
show_metrics(test_evaluator.evaluate(
        model=model,
        mapped_triples=dataset.testing.mapped_triples,
        additional_filter_triples=[
            # filtering of other positive triples
            dataset.training.mapped_triples,
            dataset.validation.mapped_triples,
        ],
    ).to_dict(),model_name,'test_metrics')
testing_evaluation_duration = time.time() - testing_evaluation_start

Test error


Evaluating on cuda:0:   0%|          | 0.00/11.2k [00:00<?, ?triple/s]

head
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.655206   0.003257     0.002442
hits_at_1                     0.652116   0.000357     0.000357
hits_at_3                     0.655151   0.001875     0.001339
hits_at_5                     0.655597   0.001964     0.001875
hits_at_10                    0.657650   0.004374     0.002500
tail
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.061445   0.007882     0.005513
hits_at_1                     0.049813   0.000357     0.000357
hits_at_3                     0.062757   0.000982     0.000625
hits_at_5                     0.063292   0.013033     0.001161
hits_at_10                    0.079986   0.014105     0.013480
both
                            optimistic  realistic  pessimistic
inverse_harmonic_mean_rank    0.358326   0.005569     0.003977
hits_at_1                     0.350964   0.000357     0.000357
hits_at_3                     0.358954  

In [20]:
infodict = {}
infodict['device'] = model.device
infodict['parameters bytes'] = model.num_parameter_bytes
infodict['number parameters'] = model.num_parameters
infodict['training duration'] = training_duration
infodict['training evaluation duration'] = training_evaluation_duration
infodict['validation evaluation duration'] = validation_evaluation_duration
infodict['testing evaluation duration'] = testing_evaluation_duration
infodict["Operating system name"] = platform.system()
infodict["Operating system version"] = platform.release()
infodict["Processor architecture"] = platform.machine()
infodict["Python version"] = sys.version
infodict["Processor model name"] = cpuinfo.get_cpu_info()['brand_raw']
infodict['Number cpu cores'] = os.cpu_count()
infodict["Total physical memory"] = psutil.virtual_memory().total

In [21]:

output = subprocess.check_output(['nvidia-smi', '--query-gpu=name', '--format=csv'])
output = output.decode('utf-8')  # convert byte string to regular string

# split output into rows and remove header row
rows = output.strip().split('\n')[1:]

# extract GPU names from each row
gpu_names = []
for row in rows:
    name = row.strip()
    gpu_names.append(name)

infodict['GPU'] = gpu_names[0]


In [22]:
infodict['loss'] = NSSALoss
infodict['num_tokens'] = num_tokens
infodict['embedding_dim'] = embedding_dim
infodict['learning_rate'] = learning_rate
infodict['optimizer'] = Adam
infodict['num_epochs'] = num_epochs
infodict['patience'] = patience
infodict['aggregation'] = 'mlp'

In [23]:
info_df = pd.DataFrame(columns=['name','value'], data = infodict.items())
info_df.to_csv(f"{model_name}/{model_name}_information.csv")
print(info_df)

                              name  \
0                           device   
1                 parameters bytes   
2                number parameters   
3                training duration   
4     training evaluation duration   
5   validation evaluation duration   
6      testing evaluation duration   
7            Operating system name   
8         Operating system version   
9           Processor architecture   
10                  Python version   
11            Processor model name   
12                Number cpu cores   
13           Total physical memory   
14                             GPU   
15                            loss   
16                      num_tokens   
17                   embedding_dim   
18                   learning_rate   
19                       optimizer   
20                      num_epochs   
21                        patience   
22                     aggregation   

                                                value  
0                              

In [24]:
def zip_folder(folder_path, output_path):
    with zipfile.ZipFile(output_path, 'w', zipfile.ZIP_DEFLATED) as zipf:
        for root, dirs, files in os.walk(folder_path):
            for file in files:
                zipf.write(os.path.join(root, file))

folder_path = model_name
output_path = f'{model_name}.zip'

zip_folder(folder_path, output_path)